In [ ]:
import pandas as pd
import re

In [ ]:
disease_codes = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/d_icd_diagnoses.csv")
patients_info = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/diagnoses_icd.csv")
pattern = r'\b(?:malignant\s+(?=.*\b(?:colon|rect|rectum|rectal|anal|anus|hepatic flexure|cecum|appendix vermiformis|flexure|large intestine)\b)|.*\b(?:colon|rect|rectum|rectal|anal|anus|hepatic flexure|cecum|appendix vermiformis|flexure|large intestine)\b\s+malignant\b)'
crc_codes = disease_codes[disease_codes["long_title"].str.contains(pattern, case=False, regex=True) & ~disease_codes["long_title"].str.contains("skin")]
# Filtering the DataFrame using isin() function
filtered_info = patients_info[patients_info['icd_code'].isin(crc_codes.icd_code.unique())]
filtered_info.subject_id.unique()

In [ ]:
genie_data = pd.read_table("/mnt/cbib/EOSC4Cancer/GENIE_CBioPortal/data_mutations_extended.txt")
genie_crc_subjects = pd.read_csv("/mnt/cbib/EOSC4Cancer/GENIE_CBioPortal/crc_public_genie_bpc_clinical_data.tsv", sep="\t")
genie_crc_subjects = genie_crc_subjects[genie_crc_subjects["Sex"]== "Male"]
genie_data = genie_data[genie_data.Tumor_Sample_Barcode.isin(genie_crc_subjects["Sample ID"].unique())]

In [ ]:
mimic_discharge_notes = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/discharge.csv")
mimic_discharge_notes = mimic_discharge_notes[mimic_discharge_notes.subject_id.isin(filtered_info.subject_id.unique())]
mimic_radiology_notes = pd.read_csv("/mnt/cbib/EOSC4Cancer/MIMIC/mimic-iv-note-deidentified-free-text-clinical-notes-2.2/note/radiology.csv")
mimic_radiology_notes = mimic_radiology_notes[mimic_radiology_notes.subject_id.isin(filtered_info.subject_id.unique())]

In [ ]:
genie_data[["Tumor_Sample_Barcode"]]

import pandas as pd
import random

# Assuming df is your DataFrame and 'Group_Column' is the column used for grouping
grouped_df = genie_data.groupby('Tumor_Sample_Barcode')

# Selecting random elements from the group
num_random_selections = 500  # Change this number to the desired number of random selections

# Select random groups (not individual elements) based on the groups in 'Group_Column'
group_keys = list(grouped_df.groups.keys())
random_group_keys = random.sample(group_keys, num_random_selections)

# Filter the original DataFrame based on the randomly selected groups
genie_result_df = pd.concat([grouped_df.get_group(key) for key in random_group_keys])


In [ ]:
import pandas as pd
import random

# Assuming df is your DataFrame and 'Group_Column' is the column used for grouping
grouped_df = mimic_discharge_notes.groupby('subject_id')

# Selecting random elements from the group
num_random_selections = 500  # Change this number to the desired number of random selections

# Select random groups (not individual elements) based on the groups in 'Group_Column'
group_keys = list(grouped_df.groups.keys())
random_group_keys = random.sample(group_keys, num_random_selections)

# Filter the original DataFrame based on the randomly selected groups
mimic_result_df = pd.concat([grouped_df.get_group(key) for key in random_group_keys])
pattern = r'Sex:\s*M'
mimic_result_df = mimic_result_df[mimic_result_df["text"].str.contains(pattern, case=False, regex=True)]
radiology_result_df = mimic_radiology_notes[mimic_radiology_notes["subject_id"].isin(mimic_result_df["subject_id"].unique())]


In [ ]:
import os

# Assuming df is your DataFrame and 'Group_Column' is the column used for grouping
clinical_grouped_df = mimic_result_df.groupby('subject_id')
radiology_grouped_df = radiology_result_df.groupby('subject_id')

all_zipped_data = []

for key, group1 in clinical_grouped_df:
    if key in radiology_grouped_df.groups:
        group2 = radiology_grouped_df.get_group(key)
        zipped_data = zip(group1['subject_id'], group1['text'], group2['text'])
        all_zipped_data.extend(list(zipped_data))
        # Instead of iterating through zipped_data here, it's better to accumulate it in all_zipped_data
names= []
# Iterate through groups and save to separate CSV files
for data in all_zipped_data:
    name, clinical_text, radiology_text = data
    names.append(name)
    new_directory = f"/mnt/cbib/EOSC4Cancer/synthetic_data/{name}/"
    os.makedirs(new_directory, exist_ok=True)
    file_name = f"/mnt/cbib/EOSC4Cancer/synthetic_data/{name}/clinical_note.txt"  # Naming files based on group names
    with open(file_name, 'w') as file:
        # Write data to the file
        file.write(clinical_text)
        file.write(radiology_text)


In [ ]:
group1

In [ ]:
grouped_df = genie_result_df.groupby("Tumor_Sample_Barcode")
for name, (_, row) in zip(names, grouped_df):
    file_name = f"/mnt/cbib/EOSC4Cancer/synthetic_data/{name}/mutations_data.csv"
    row.to_csv(file_name)

In [ ]:
import re
# remove annoying characters
chars = {
    '\xc2\x82' : ',',        # High code comma
    '\xc2\x84' : ',,',       # High code double comma
    '\xc2\x85' : '...',      # Tripple dot
    '\xc2\x88' : '^',        # High carat
    '\xc2\x91' : '\x27',     # Forward single quote
    '\xc2\x92' : '\x27',     # Reverse single quote
    '\xc2\x93' : '\x22',     # Forward double quote
    '\xc2\x94' : '\x22',     # Reverse double quote
    '\xc2\x95' : ' ',
    '\xc2\x96' : '-',        # High hyphen
    '\xc2\x97' : '--',       # Double hyphen
    '\xc2\x99' : ' ',
    '\xc2\xa0' : ' ',
    '\xc2\xa6' : '|',        # Split vertical bar
    '\xc2\xab' : '<<',       # Double less than
    '\xc2\xbb' : '>>',       # Double greater than
    '\xc2\xbc' : '1/4',      # one quarter
    '\xc2\xbd' : '1/2',      # one half
    '\xc2\xbe' : '3/4',      # three quarters
    '\xca\xbf' : '\x27',     # c-single quote
    '\xcc\xa8' : '',         # modifier - under curve
    '\xcc\xb1' : '',          # modifier - under line
    '\x95' : ' ',
}
def replace_chars(match):
    char = match.group(0)
    return chars[char]

def remove_annoying_chars(text):
    pattern = '(' + '|'.join(chars.keys()) + ')'
    return re.sub(pattern, replace_chars, text)

# Example usage:
input_text = "Your input text here with annoying characters..."
result_text = remove_annoying_chars(input_text)
print(result_text)

In [ ]:
remove_annoying_chars(mimic_discharge_notes.iloc[10]["text"])

In [ ]:
mimic_discharge_notes.iloc[10]["text"].replace("\n\n", "").replace("\n", "").replace("___", "")

In [ ]:
from preprocessing_utils import split_text_to_sentences

In [ ]:
import utils
REGEX_FILE  = utils.load_regex_patterns("../data/regex_patterns.json")
wh = split_text_to_sentences(mimic_discharge_notes.iloc[0]["text"].replace("\n \n", "\n\n").replace("_", "\n\n").replace("===", '\n\n'), REGEX_FILE)

In [8]:
with open('/mnt/cbib/EOSC4Cancer/synthetic_data/10669816/clinical_note.txt') as f:
    lines = f.read()

In [9]:
lines

' \nName:  ___                Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: SURGERY\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___\n \n___ Complaint:\nColon cancer\n \nMajor Surgical or Invasive Procedure:\n___: Laparoscopic right colectomy\n\n \nHistory of Present Illness:\n___ year old female with PMHx pertinent for HTN, stroke at age \n___,\nsmoking, who presents for initial evaluation due to colon mass\nfound on colonoscopy 2 months ago. Patient mentions that she was\nbeing worked up for cataract surgery when she was found to be\nanemic. In consequence of this finding she underwent colonoscopy\nand was found to have a 4 cm mass in the ascending colon with\nluminal narrowing, question for local spread beyond the colon \nand\nfor a nodule in inferior right lobe of the liver. Since then,\npatient underwent MRI, CT and PET scan. PET scan showed \nmalignant\nmass in as

In [1]:
import preprocess_clinical_notes

In [2]:
df = preprocess_clinical_notes.tokenize_clinical_note(id_list=["10001860"], source_directory="/mnt/cbib/EOSC4Cancer/synthetic_data")

In [3]:
for _, row in df.iterrows():
    print(row["sentence"])






 Sex:   M
 ORTHOPAEDICS
 Sulfa (Sulfonamide Antibiotics) / Penicillins
 
 neck pain s/p fall
 None on this Admission

male transferred from outside hospital for  evaluation of cervical
fracture
Today the patient was  attempting to use the bathroom and bent forward and fell hitting  the back of his head
There was no loss of consciousness
The  patient complains of headache and neck pain
The outside  hospital the patient had the head laceration stapled
A CT scan  did demonstrate the fracture
The patient denies any numbness  tingling in his arms or legs
No weakness in his arms or legs
Denies any bowel incontinence or bladder retention
No saddle  anesthesia
Denies any chest pain shortness of breath or  abdominal pain

 a. fib colon ca htn copd
 warfarin allopurinol asacol
 pcn sulfa

 NC

UEC5C6C7C8T1 (lat arm)  (thumb) (mid fing) (sm finger) (med arm) Rintact      intact        intact        intact        intact Lintact      intact        intact        intact        intact
T2-L1 (Trunk

In [4]:
df.head(50)

,id,field,sentence,IsTitle,IsInlineTitle
0,10001860,Name,,True,False
1,10001860,Unit No,,True,False
2,10001860,Admission Date,,True,False
3,10001860,Discharge Date,,True,False
4,10001860,Date of Birth,Sex: M,False,True
5,10001860,Service,ORTHOPAEDICS,False,True
6,10001860,Allergies,Sulfa (Sulfonamide Antibiotics) / Penicillins,False,True
7,10001860,Attending,,False,True
8,10001860,Chief Complaint,neck pain s/p fall,False,True
9,10001860,Major Surgical or Invasive Procedure,None on this Admission,False,True


In [7]:
import re
text = "This is a\nsample text\nwith\nnewline\ncharacters."
modified_text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
print(modified_text)


This is a sample text

with newline characters.
